In [1]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [1]:
import panel as pn
from dotenv import load_dotenv

load_dotenv()

pn.extension()

In [2]:
import param

from pyllments.base.model_base import Model
from pyllments.payloads.message import MessagePayload, MessageModel

class ChatInterfaceModel(Model):
    # TODO: Implement batch interface for messages - populating message_list > iterating
    message_list = param.List(instantiate=True)
    persist = param.Boolean(default=False, instantiate=True)
    new_message = param.ClassSelector(class_=MessagePayload)
    
    def __init__(self, **params):
        super().__init__(**params)

        self._create_watchers()

    def _create_watchers(self):
        self.param.watch(self._new_message_updated, 'new_message', precedence=10)

    def _new_message_updated(self, event):
        if self.new_message.model.mode == 'stream':
            self.new_message.model.stream()
        self.message_list.append(self.new_message)


In [3]:
async def generate_messages():
    """
    Generator to yield messages from the message list.
    """
    for message in [1,2,3]:
        yield message

type(generate_messages())






async_generator

In [4]:
import panel as pn
import param

from pyllments.base.element_base import Element
from pyllments.base.model_base import Model
# from pyllments.elements.chat_interface import ChatInterfaceModel
from pyllments.payloads.message import MessagePayload
from langchain_core.messages.base import BaseMessage
from langchain_core.messages.human import HumanMessage

class ChatInterfaceElement(Element):
    """
    Model:
    - messages in the chat
    - message input
    Views:
    - chat feed: 
    - chat input
    - send button:
    Ports:
    - input:
        - message_input: MessagePayload
    - output port
        - message_output: MessagePayload
    """
    model = param.ClassSelector(
        class_=Model,
        is_instance=True
    )
    model_params = param.Dict(default={})

    chatfeed_view = param.ClassSelector(class_=pn.chat.ChatFeed, is_instance=True)
    chat_input_view = param.ClassSelector(class_=pn.chat.ChatAreaInput, is_instance=True)
    send_button_view = param.ClassSelector(class_=pn.widgets.Button, is_instance=True)

    def __init__(self, persist=False, **params):
        super().__init__(**params)
        self.model = ChatInterfaceModel(**self.model_params)
        
        self.message_output_setup()
        self.message_input_setup()

    def message_output_setup(self):
        """Sets up the output message port"""
        def pack(new_message: MessagePayload) -> MessagePayload:
            return new_message

        self.ports.add_output(
            name='message_output',
            pack_payload_callback=pack)
    
    def message_input_setup(self):
        """Sets up the input message port"""
        def unpack(payload: MessagePayload):
            self.model.new_message = payload
        
        self.ports.add_input(
            name='message_input',
            unpack_payload_callback=unpack)

    def create_chatfeed_view(self, **kwargs):
        """
        Creates and returns a new instance of the chatfeed whichi
        contains the visual components of the message payloads.
        """
        if self._view_exists(self.chatfeed_view):
            return self.chatfeed_view
        # When first loaded
        self.chatfeed_view = pn.chat.ChatFeed(**kwargs)
        message_views = [
            message.create_message_view() 
            for message in self.model.message_list
        ]
        self.chatfeed_view.extend(message_views)

        def _update_chatfeed(event):
            self.chatfeed_view.append(event.new.create_message_view())
        # This watcher should be called before the payload starts streaming.
        self.model.param.watch(_update_chatfeed, 'new_message', precedence=0)
        return self.chatfeed_view


    def create_chat_input_view(self, **kwargs):
        """
        Creates and returns a new instance of ChatAreaInput view.
        """
        if self._view_exists(self.chat_input_view):
            return self.chat_input_view

        self.chat_input_view = pn.chat.ChatAreaInput(
            placeholder='Enter your message',
            **kwargs)
        self.chat_input_view.param.watch(self._on_send, 'value')
        return self.chat_input_view
    

    def create_send_button_view(self, **kwargs):
        """
        Creates and returns a new instance of Button view for sending messages.
        """
        if self._view_exists(self.send_button_view):
            return self.send_button_view

        self.send_button_view = pn.widgets.Button(name='send', **kwargs)
        self.send_button_view.on_click(self._on_send)

        return self.send_button_view
    
    @Element.port_stage_emit_if_exists('message_output', 'new_message')
    def _on_send(self, event):
        """
        Handles the send button event by appending the user's message to the chat model,
        clearing the input field, and updating the chat feed view.
        """
        
        if event.obj is self.send_button_view: # When send button is clicked
            if self.chat_input_view:
                input_text = self.chat_input_view.value_input
                self.chat_input_view.value_input = ''
                new_message = MessagePayload(
                    message=HumanMessage(input_text),
                    mode='atomic')
            self.model.new_message = new_message
            
        elif event.obj is self.chat_input_view: # When value changes on 'enter'
            input_text = self.chat_input_view.value
            new_message = MessagePayload(
                message=HumanMessage(input_text),
                mode='atomic')
            self.model.new_message = new_message


In [5]:
# chat_interface_element = ChatInterfaceElement()
# chat_input_view = chat_interface_element.create_chat_input_view()
# # chat_input_view.value = 'hello'

# send_button_view = chat_interface_element.create_send_button_view()
# pn.Row(chat_input_view, send_button_view).servable()

In [6]:
import param

from langchain_core.language_models import BaseLanguageModel
from langchain_core.messages.base import BaseMessage
from langchain_core.messages.ai import AIMessage
from langchain_openai import ChatOpenAI

from pyllments.base.model_base import Model

class LLMChatModel(Model):
    chat_model = param.ClassSelector(class_=BaseLanguageModel, doc="""
        Instance of active chat model.""")

    model_args = param.Dict(default={}, doc="""
        Takes a dictionary of arguments to pass to expose and send to the
        model class. If you set a None value as the key, the argument will be exposed,
        with the default value set. Passing a list is the same as passing a dict
        with all values set to None.""") # TODO Allow nested dict for model_name: model_args format
    
    provider_name = param.String(default='openai', doc='Provider of the model')
    model_name = param.String(default='gpt-3.5-turbo', doc='Name of the model')
    incoming_messages = param.List(item_type=BaseMessage)
    output_mode = param.Selector(
        objects=['atomic', 'stream'],
        default='stream',
        )
    # new_message = param.ClassSelector(class_=AIMessage)
    outgoing_message = param.Parameter(doc="""
        AIMessage or Stream, depending on output_mode""")

    def __init__(self, **params):
        super().__init__(**params)
        self._initialize_provider()
        self._initialize_model()
        # self._set_params()
    #     self._create_watchers()

    # def _create_watchers(self):
    #     self.param.watch(
    #         self._new_outgoing_message,
    #         'outgoing_message',
    #         onlychanged=False
    #         )
    # def _set_params(self):
    #     """Sets specified model_args as params of the object"""
    #     if self.model_args:
    #         for arg, val in self.model_args.items():
    #             if arg in self.model_class.__fields__:
    #                 if val is None:
    #                     default = self.model_class.__fields__[arg].default
    #                     self.model_args[arg] = default
    #                     self.param.add_parameter(arg, param.Parameter(default, per_instance=True))
    #                 else:
    #                     self.param.add_parameter(arg, param.Parameter(val, per_instance=True))
    #                 # self.model_args_list.append(arg)
    #             else:
    #                 raise ValueError(f"'{arg}' is missing from the model class's signature")
    #             self.param.watch(self._create_model, [*self.model_args.keys()])


    # def _create_model(self, event=None):
    #     """Creates the model instance on init and when any of the parameters change"""
    #     arg_vals = {arg: self.param.values()[arg] for arg in self.model_args.keys()}
    #     self.model = self.model_class(**arg_vals)
    
    def _initialize_provider(self):
        """Initializes the provider"""
        match self.provider_name:
            case 'openai':
                from langchain_openai import ChatOpenAI
                self.provider = ChatOpenAI
            case 'anthropic':
                from langchain_anthropic import ChatAnthropic
                self.provider = ChatAnthropic
            case 'groq':
                from langchain_groq import ChatGroq
                self.provider = ChatGroq
            case 'mistral':
                from langchain_mistralai import ChatMistralAI
                self.provider = ChatMistralAI
            case 'google':
                from langchain_google_genai import ChatGoogleGenerativeAI
                self.provider = ChatGoogleGenerativeAI
            case _:
                raise ValueError(f"Provider name '{self.provider_name}' is not valid")

    def _initialize_model(self):
        """Initializes the model"""
        self.chat_model = self.provider(model_name=self.model_name, **self.model_args)




In [7]:
from typing import AsyncGenerator, Generator

import param
import panel as pn
from langchain_core.messages import AIMessage

from pyllments.base.element_base import Element
from pyllments.payloads.message import MessagePayload
# from pyllments.elements.llm_chat import LLMChatModel

class LLMChatElement(Element):
    model = param.ClassSelector(class_=Model)
    model_params = param.Dict(default={})
  
    def __init__(self, **params):
        super().__init__(**params)
        self.model = LLMChatModel(**self.model_params)

        self._message_output_setup()
        self._messages_input_setup()

        self._create_watchers()
        
    def _message_output_setup(self):
        if self.model.output_mode == 'stream':
            def pack(outgoing_message: AsyncGenerator | Generator) -> MessagePayload:
                payload = MessagePayload(
                    message_type='ai',
                    message_stream=outgoing_message,
                    mode='stream'
                )
                return payload
        elif self.model.output_mode == 'atomic':
            def pack(outgoing_message: AIMessage) -> MessagePayload:
                payload = MessagePayload(
                    message=outgoing_message,
                    mode='atomic'
                )
                return payload
            
        self.ports.add_output(name='message_output', pack_payload_callback=pack)

    def _messages_input_setup(self):
        def unpack(payload: MessagePayload):
            if payload.model.mode == 'atomic':
                if self.model.output_mode == 'atomic':
                    self.model.outgoing_message = self.model.chat_model.invoke(
                        [payload.model.message]
                    )
                elif self.model.output_mode == 'stream':
                    self.model.outgoing_message = self.model.chat_model.stream(
                        [payload.model.message]
                    )
            elif payload.model.mode == 'batch':
                if self.model.output_mode == 'atomic':
                    self.model.outgoing_message = self.model.chat_model.invoke(
                        payload.model.messages_batch
                    )
                elif self.model.output_mode == 'stream':
                    self.model.outgoing_message = self.model.chat_model.stream(
                        payload.model.messages_batch
                    )
        self.ports.add_input(name='messages_input', unpack_payload_callback=unpack)

    def _create_watchers(self):
        self.model.param.watch(self._outgoing_message_updated, 'outgoing_message')
    
    @Element.port_stage_emit_if_exists('message_output', 'outgoing_message')
    def _outgoing_message_updated(self, event):
        pass


    # def create_temperature_view(self, **kwargs):
    #     self.temperature_view = pn.widgets.FloatSlider(**kwargs)
    #     return self.temperature_view

In [8]:
# chat_interface = ChatInterfaceElement()

# llm_chat = LLMChatElement(model_params={'output_mode': 'atomic'})

# chat_interface.ports.output['message_output'] > llm_chat.ports.input['messages_input']
# # chat_interface.model.new_message = MessagePayload(
# #     message=HumanMessage('hello'),
# #     mode='atomic'
# # )
# chat_interface.ports.output['message_output'].stage_emit(
#     message_payload=MessagePayload(
#         message=HumanMessage('hello'),
#         mode='atomic'
# ))

# llm_chat.model.outgoing_message

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-779fc32a-3f73-454d-b113-7b8e095cec02-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

In [8]:
# chat_interface = ChatInterfaceElement()

# llm_chat = LLMChatElement(model_params={'output_mode': 'stream'})

# chat_interface.ports.output['message_output'] > llm_chat.ports.input['messages_input']
# # chat_interface.model.new_message = MessagePayload(
# #     message=HumanMessage('hello'),
# #     mode='atomic'
# # )
# chat_interface.ports.output['message_output'].stage_emit(
#     message_payload=MessagePayload(
#         message=HumanMessage('hello'),
#         mode='atomic'
# ))

# llm_chat.model.outgoing_message

<generator object BaseChatModel.stream at 0x7fc402fa18b0>

In [8]:
chat_interface = ChatInterfaceElement()

llm_chat = LLMChatElement(model_params={'output_mode': 'stream'})

chat_interface.ports.output['message_output'] > llm_chat.ports.input['messages_input']
llm_chat.ports.output['message_output'] > chat_interface.ports.input['message_input']


# chat_interface.ports.output['message_output'].stage_emit(
#     message_payload=MessagePayload(
#         message=HumanMessage('hello'),
#         mode='atomic'
# ))

# llm_chat.model.outgoing_message

In [21]:
def serve(panel_obj):
    # pn.state.kill_all_servers()
    pn.serve(panel_obj, port=4321)

In [11]:
chatfeed_view = chat_interface.create_chatfeed_view()
send_button_view = chat_interface.create_send_button_view()
chat_input_view = chat_interface.create_chat_input_view()

pn.Column(
    chatfeed_view,
    pn.Row(chat_input_view, send_button_view)
).servable()


/workspaces/pyllments/pyllments/base/element_base.py:20: UserWarning: ChatFeed(_placeholder=ChatMessage, sizing_mode='stretch_width')
    [0] Row
        [0] Markdown(str)
    [1] Row
        [0] Markdown(str) already exists. Returning existing view.
  warnings.warn(f'{view} already exists. Returning existing view.')
/workspaces/pyllments/pyllments/base/element_base.py:20: UserWarning: Button(clicks=1, name='send') already exists. Returning existing view.
  warnings.warn(f'{view} already exists. Returning existing view.')
/workspaces/pyllments/pyllments/base/element_base.py:20: UserWarning: ChatAreaInput(placeholder='Enter your message') already exists. Returning existing view.
  warnings.warn(f'{view} already exists. Returning existing view.')


BokehModel(combine_events=True, render_bundle={'docs_json': {'abedc556-f22a-48ff-ac67-e315007b04cc': {'version…